In [48]:
import pandas as pd
import os


df = pd.read_csv("DKSalaries.csv")
df = df[["Roster Position", "Name + ID", "Salary", "TeamAbbrev"]]


current_directory = os.getcwd()

# Construct the path to the file in the higher directory
higher_directory = os.path.abspath(
    os.path.join(current_directory, os.pardir, os.pardir)
)
file_in_higher_directory_path = os.path.join(higher_directory, "players.csv")
players = pd.read_csv(file_in_higher_directory_path)

In [49]:
players.head(5)

,player_id,full_name,position
0,1630173,Precious Achiuwa,Forward
1,203500,Steven Adams,Center
2,1628389,Bam Adebayo,Center-Forward
3,1630534,Ochai Agbaji,Guard
4,1630583,Santi Aldama,Forward-Center


In [50]:
def appFunc_getID(nameAndID):
    draftking_full_name = nameAndID.split("(")[0].strip().lower()
    filtered_df = players[players["full_name"].str.lower() == draftking_full_name]

    if filtered_df.empty:
        print("No row found for", draftking_full_name)
        return None

    return int(filtered_df.iloc[0]["player_id"])


df["player_id"] = df["Name + ID"].apply(appFunc_getID)
df.head(5)

,Roster Position,Name + ID,Salary,TeamAbbrev,player_id
0,PF/C/F/UTIL,Giannis Antetokounmpo (32200475),11700,MIL,203507
1,C/UTIL,Joel Embiid (32200486),11500,PHI,203954
2,PG/G/UTIL,Damian Lillard (32200421),8800,MIL,203081
3,C/UTIL,Alperen Sengun (32200611),8600,HOU,1630578
4,PG/G/UTIL,Tyrese Maxey (32200576),8300,PHI,1630178


In [52]:
salary_dict = {}
team_dict = {}

for _, row in df.iterrows():
    salary_dict[row["player_id"]] = row["Salary"]
    team_dict[row["player_id"]] = row["TeamAbbrev"]

In [ ]:
# 1. Make dict of roster positions with players as array
# 2. Generate N random valid lineups
# 3. Simulate, get overall rankings. Keep top 5-10% just as they are (elitism). Use top 25% as sure parents, with rest having 33%?
# 4. Inherit by:
# for each survivor, find M other random survivors. Make X children by randomly deciding traits from parents
# 5. Mutate children by:
# for each child, randomly swap out T players, with T changing over time?
# 6. Add Q purely random lineups
# 6. Save best, repeat 2-6 until satisfied

### 1. Make dict of roster positions with players as array

In [46]:
player_dict = {}
player_dict["PG"] = []
player_dict["SG"] = []
player_dict["SF"] = []
player_dict["PF"] = []
player_dict["C"] = []
player_dict["G"] = []
player_dict["F"] = []
player_dict["UTIL"] = []

for _, row in df.iterrows():
    if "PG" in row["Roster Position"]:
        player_dict["PG"].append(row["player_id"])
    if "SG" in row["Roster Position"]:
        player_dict["SG"].append(row["player_id"])
    if "SF" in row["Roster Position"]:
        player_dict["SF"].append(row["player_id"])
    if "PF" in row["Roster Position"]:
        player_dict["PF"].append(row["player_id"])
    if "C" in row["Roster Position"]:
        player_dict["C"].append(row["player_id"])
    if "G" in row["Roster Position"]:
        player_dict["G"].append(row["player_id"])
    if "F" in row["Roster Position"]:
        player_dict["F"].append(row["player_id"])
    if "UTIL" in row["Roster Position"]:
        player_dict["UTIL"].append(row["player_id"])

len(player_dict["UTIL"])

72

In [78]:
from operator import itemgetter


def lineup_is_valid(lineup_array):
    """Assumes that the array of id's has one per correct slot."""
    if len(lineup_array) != 8:  # check is right length
        return False
    if len(set(lineup_array)) != len(lineup_array):  # check for dupes
        return False
    salaries = itemgetter(*lineup_array)(salary_dict)
    if sum(salaries) > 50000:  # check in budget
        return False
    teams = itemgetter(*lineup_array)(team_dict)
    if len(set(teams)) < 2:  # Check if has both teams
        return False
    return True

### 2. Generate N random valid lineups

In [82]:
import math
import random


def generate_n_pure_random_lineups(gen):
    lineups = []
    max_length = math.floor((1 / gen) * 100)
    print(max_length)
    while len(lineups) < max_length:
        pg = random.choice(player_dict["PG"])
        sg = random.choice(player_dict["SG"])
        sf = random.choice(player_dict["SF"])
        pf = random.choice(player_dict["PF"])
        c = random.choice(player_dict["C"])
        g = random.choice(player_dict["G"])
        f = random.choice(player_dict["F"])
        util = random.choice(player_dict["UTIL"])
        new_lineup = [pg, sg, sf, pf, c,g,f,util]
        if lineup_is_valid(new_lineup):
            lineups.append(new_lineup)
    return lineups

generate_n_pure_random_lineups(69)

1


[[203081, 1627788, 1641715, 202699, 203954, 1641708, 1626162, 1630699]]

### Simulate, get overall rankings. Keep top 5-10% just as they are (elitism). Use top 25% as sure parents, with rest having 33%?